In [ ]:
import tensorflow as tf
%load_ext tensorboard.notebook

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Softmax, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import datetime
from utils import sparse_categorical_crossentropy_with_logits

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# train_dir = os.path.join('..', 'detect_classify_images', 'failed')
train_dir = os.path.join('images', 'train')
valid_dir = os.path.join('images', 'validation')

class_dirs = os.listdir(train_dir)
train_class_dirs = [os.path.join(train_dir, class_dir) for class_dir in class_dirs]
valid_class_dirs = [os.path.join(valid_dir, class_dir) for class_dir in class_dirs]

In [ ]:
num_class_train = [len(os.listdir(train_class_dir)) for train_class_dir in train_class_dirs]
num_class_valid = [len(os.listdir(valid_class_dir)) for valid_class_dir in valid_class_dirs]

total_train = sum(num_class_train)
total_valid = sum(num_class_valid)

In [ ]:
print("Total training images:", total_train)
print("Total validation images:", total_valid)

In [ ]:
batch_size = 128
epochs = 10
HEIGHT = 75
WIDTH = 75

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255)
valid_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir,
    shuffle=True,
    target_size=(HEIGHT, WIDTH),
    class_mode='sparse'
)

In [ ]:
valid_data_gen = valid_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=valid_dir,
    target_size=(HEIGHT, WIDTH),
)

In [ ]:
sample_training_images, _ = next(train_data_gen)

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:5])

In [ ]:
# model = Sequential([
#     Conv2D(16, 3, padding='same', activation='relu', input_shape=(HEIGHT, WIDTH, 3)),
#     MaxPooling2D(),
# #     BatchNormalization(),
#     Conv2D(32, 3, padding='same', activation='relu'),
#     MaxPooling2D(),
# #     BatchNormalization(),
#     Conv2D(64, 3, padding='same', activation='relu'),
#     MaxPooling2D(),
# #     BatchNormalization(),
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dense(2),
#     Softmax()
# ])

model = Sequential([
    Flatten(input_shape=(HEIGHT, WIDTH, 3)),
    Dense(128, activation='relu'),
    Dense(3),
    Softmax()
])

# model = tf.keras.models.load_model('model_new3')

In [ ]:
model.layers[0]

In [ ]:
model.compile(optimizer='adam',
              loss=sparse_categorical_crossentropy_with_logits,
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit_generator(
    train_data_gen,
#     steps_per_epoch=math.ceil(total_train // batch_size),
    epochs=10,
#     validation_data=valid_data_gen,
#     validation_steps=total_valid,
#     verbose=1
#     callbacks=[tensorboard_callback]
)

In [ ]:
# %tensorboard --logdir logs/fit

In [ ]:
# acc = history.history['acc']
# # val_acc = history.history['val_acc']

# loss=history.history['loss']
# # val_loss=history.history['val_loss']

# epochs_range = range(epochs)

# plt.figure(figsize=(8, 8))
# plt.subplot(1, 2, 1)
# plt.plot(epochs_range, acc, label='Training Accuracy')
# # plt.plot(epochs_range, val_acc, label='Validation Accuracy')
# plt.legend(loc='lower right')
# plt.title('Training and Validation Accuracy')

# plt.subplot(1, 2, 2)
# plt.plot(epochs_range, loss, label='Training Loss')
# # plt.plot(epochs_range, val_loss, label='Validation Loss')
# plt.legend(loc='upper right')
# plt.title('Training and Validation Loss')
# plt.show()

In [ ]:
model.save('model_new3')

In [ ]:
model.get_weights()

In [ ]:
labels = ['point_foreward', 'point_up']

In [ ]:
model2 = tf.keras.models.load_model('model')

In [ ]:
model2.get_weights()

In [ ]:
test_label_ind = 0
test_dir = 'images/train/{}'.format(labels[test_label_ind])
# test_dir = os.path.join(train_dir, 'point_up')
files = os.listdir(test_dir)
# print(files)
for f in files:
    img = cv2.imread(os.path.join(test_dir, f))
    img = cv2.resize(img, (HEIGHT, WIDTH))
    batch = np.expand_dims(img, axis=0).astype(np.float32)
    res = model.predict(batch)
    res2 = model2.predict(batch)
    label = labels[np.argmax(res[0])]
#     print(label)
    print('before: {} | now: {}'.format(res2, res))

In [ ]:
# # test_dir = 'images/train/point_foreward'
# test_dir = os.path.join('..', 'detect_classify_images', 'failed', 'point_foreward')
# # test_dir = os.path.join(train_dir, 'point_up')
# files = os.listdir(test_dir)
# imgs = []
# for f in files:
#     img = cv2.imread(os.path.join(test_dir, f))
#     img = cv2.resize(img, (HEIGHT, WIDTH))
# #     batch = np.expand_dims(img, axis=0).astype(np.float32)
#     imgs.append(img)
# y = np.array([0 for i in range(len(files))])

In [ ]:
# model.evaluate(np.array(imgs), y)


In [ ]:
# model.save('model')

In [ ]:
# imgs2 = np.array([imgs[0]])
# y2 = np.array([1])
# print(y2)
# model.evaluate(imgs2, y2)
# # ins = [imgs2, y2, np.ones(2), 1]


In [ ]:
# model.fit(imgs, y, epochs=10)